In [16]:
import pandas as pd
import subprocess

In [17]:
# Load the CSV file
df = pd.read_csv('../../results/compare_human_mouse_mxra8/site_binding.csv')

# Filter: keep only rows of mouse_Mxra8
df = df[df['ligand'] == 'mouse_Mxra8']

# Select and clean relevant columns
columns_to_keep = ['protein_site', 'summed_effect', '6nk7_chain']
new_df = df[columns_to_keep].copy()

# Drop rows where 'protein_site' is missing or not finite
new_df = new_df.dropna(subset=['protein_site'])

# Convert 'protein_site' to integer
new_df['protein_site'] = new_df['protein_site'].astype(int)

# Save the cleaned DataFrame
output_path = '../../manual_analyses/chimera_structures/results/mMxra8_binding_sum.csv'
new_df.to_csv(output_path, index=False)

# Optional: confirm success
print(f"Saved {len(new_df)} rows to {output_path}")


Saved 937 rows to ../../manual_analyses/chimera_structures/results/mMxra8_binding_sum.csv


In [18]:
# prepare data in chimera format
def aggregate_binding_mean(infile, name, outfile):
    tmp_df = pd.read_csv(infile)
    tmp_df = tmp_df.round(3)

    # Modify the dataframe to prepend a tab character and format as strings
    tmp_df["protein_site"] = tmp_df["protein_site"].astype(str)
    tmp_df["summed_effect"] = tmp_df["summed_effect"].astype(str)
    tmp_df["formatted"] = "\t" + "/"  + tmp_df['6nk7_chain'] + ":" + tmp_df["protein_site"] + "\t" + tmp_df["summed_effect"]

    with open(outfile, "w") as f:
        # Write header lines
        f.write(f"attribute: {name}\n")
        f.write("match mode: any\n")
        f.write("recipient: residues\n")

    # Append the tab separated dataframe to the file without quotes and with an escape character
    tmp_df["formatted"].to_csv(outfile, sep="\t", index=False, header=False, mode="a")
    subprocess.run(["sed", "-i", 's/"//g', outfile], check=True)

# Call function above
aggregate_binding_mean('../../manual_analyses/chimera_structures/results/mMxra8_binding_sum.csv', "binding", '../../manual_analyses/chimera_structures/results/mMxra8_binding_sum.defattr')
